In [ ]:
from wit_tooling import load_wit_s3, plot_to_png, set_output_name
import boto3
import fiona
import awswrangler as wr

In [ ]:
session = boto3.Session(profile_name='dev')
s3_client = session.client('s3')

In [ ]:
db_name = 'wit_test'
print(wr.catalog.tables(database=db_name, boto3_session=session))
if db_name not in wr.catalog.databases(boto3_session=session).values:
    wr.catalog.create_database(db_name, boto3_session=session)
table_name = 'data_lock_4'
wit_path = "s3://dea-wit-dev/c3-samples-3577/lock_4/"
wr.s3.store_parquet_metadata(path=wit_path, database=db_name, table=table_name, dataset=False, mode='overwrite', boto3_session=session)

In [ ]:
shapefile = './shapefiles/Lock_4_Floodplain_25062021.shp'
bucket_name = 'dea-wit-dev'
key_prefix = 'delivery/lock_4/floodplain/'
s3_path = 's3://' + '/'.join([bucket_name, key_prefix])

In [ ]:
with fiona.open(shapefile) as allshapes:
    for shape in iter(allshapes):
        img_name = set_output_name(shape, ['ID'])
        wit_data = load_wit_s3(shape['geometry'], db_name, table_name, session)
        b_img = plot_to_png(wit_data.to_numpy(), img_name)
        s3_client.put_object(Body=b_img, Bucket=bucket_name, Key=key_prefix+img_name+'.png')
        wr.s3.to_csv(wit_data, path=s3_path+img_name+'.csv', boto3_session=session)

In [ ]:
wr.s3.list_objects('s3://dea-wit-dev/delivery/', boto3_session=session)